# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv, find_dotenv


%load_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
# this will load all the envars from a .env file located in the project root (api)
%load_ext dotenv
load_dotenv(find_dotenv())

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
PG_USER=os.getenv("PG_USER", "")
PG_PASSWORD=os.getenv("PG_PASSWORD", "")
PG_HOST=os.getenv("PG_HOST", "")
PG_PORT=os.getenv("PG_PORT", "5432")
PG_DB_NAME=os.getenv("PG_DB_NAME", "")
PG_SOCKET=os.getenv("PG_SOCKET", "")

if PG_SOCKET != "":
    import pg8000
    connect_to_db = f"postgresql+pg8000://{PG_USER}:{PG_PASSWORD}@/{PG_DB_NAME}?unix_sock={PG_SOCKET}/.s.PGSQL.5432"
else:
    connect_to_db = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB_NAME}"

print(connect_to_db)
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

monthly total before running time.

In [ ]:
%%sql stat_incorp_filings_daily_completed  <<
SELECT b.identifier AS INCORPORATION_NUMBER
       , b.legal_name AS INCORPORATION_NAME
       , u.username AS FILING_USER
       , f.status 
       , f.filing_date at time zone 'utc' at time zone 'pst' AS FILING_TIMESTAMP_PST
       , f.effective_date at time zone 'utc' at time zone 'pst' AS EFFECTIVE_TIMESTAMP_PST
FROM businesses b, filings f, users u 
WHERE b.id = f.business_id 
AND f.filing_type='incorporationApplication'
AND f.submitter_id=u.id 
AND date(f.filing_date at time zone 'utc' at time zone 'pst') = date(current_date-1)
ORDER BY FILING_TIMESTAMP_PST;

In [ ]:
incorp_edt = stat_incorp_filings_daily_completed.DataFrame()

datestr = datetime.strftime(datetime.now()-timedelta(1), "%Y-%m-%d")
filename =  os.path.join(os.getcwd(), r"data/")+"incorporation_filings_daily_stats_" + datestr +".csv"

with open(filename, "w") as f:
    if incorp_edt.empty:
        f.write("No Data Retrieved for Incorporation Application on " + datestr)         
    else:  
        f.write("Incorporation Application(s) on " + datestr + ":\n")            
        incorp_edt.to_csv(f, sep=",", encoding="utf-8", index=False)  
    f.write("\n\n")   

In [ ]:
%%sql stat_benefit_filings_daily_completed  <<
SELECT b.identifier AS INCORPORATION_NUMBER
       , b.legal_name AS INCORPORATION_NAME
       , u.username AS FILING_USER
       , f.status 
       , f.filing_date at time zone 'utc' at time zone 'pst' AS FILING_TIMESTAMP_PST
       , f.effective_date at time zone 'utc' at time zone 'pst' AS EFFECTIVE_TIMESTAMP_PST
FROM businesses b, filings f, users u 
WHERE b.id = f.business_id 
AND b.legal_type='BEN'
AND f.filing_type='alteration'
AND f.submitter_id=u.id 
AND date(f.filing_date at time zone 'utc' at time zone 'pst') = date(current_date-1)
ORDER BY FILING_TIMESTAMP_PST;

In [ ]:
benefit_edt = stat_benefit_filings_daily_completed.DataFrame()

with open(filename, "a") as f:  
    if benefit_edt.empty:
        f.write("No Data Retrieved for Alterations to Benefit Company on " + datestr)         
    else: 
        f.write("Alterations to Benefit Company on " + datestr + ":\n")  
        benefit_edt.to_csv(f, sep=",", encoding="utf-8", index=False) 
    f.write("\n\n")  

In [ ]:
%%sql stat_bc_company_filings_daily_completed  <<
SELECT b.identifier AS INCORPORATION_NUMBER
       , b.legal_name AS INCORPORATION_NAME
       , u.username AS FILING_USER
       , f.status 
       , f.filing_date at time zone 'utc' at time zone 'pst' AS FILING_TIMESTAMP_PST
       , f.effective_date at time zone 'utc' at time zone 'pst' AS EFFECTIVE_TIMESTAMP_PST
FROM businesses b, filings f, users u 
WHERE b.id = f.business_id 
AND b.legal_type='BC'
AND f.filing_type='alteration'
AND f.submitter_id=u.id 
AND date(f.filing_date at time zone 'utc' at time zone 'pst') = date(current_date-1)
ORDER BY FILING_TIMESTAMP_PST;

In [ ]:
bc_company_edt = stat_bc_company_filings_daily_completed.DataFrame()

with open(filename, "a") as f:  
    if bc_company_edt.empty:
        f.write("No Data Retrieved for Alterations to BC Limited Company on " + datestr)         
    else: 
        f.write("Alterations to BC Limited Company on " + datestr + ":\n")  
        bc_company_edt.to_csv(f, sep=",", encoding="utf-8", index=False) 
    f.write("\n\n")    

In [ ]:
%%sql stat_benefit_corporations_total_completed  <<
SELECT count(*) 
FROM businesses b
WHERE b.legal_type='BEN';

In [ ]:
with open(filename, "a") as f:  
    f.write("The Total Number of Benefit Companies to Date:\n")
    stat_benefit_corporations_total_completed.DataFrame().to_csv(f, sep=",", encoding="utf-8", index=False) 